In [1]:
import numpy as np
import pandas as pd
# import lightgbm as lbm
import xgboost as xgb
import optuna

In [2]:
data = pd.read_csv('final_df_start_finish.csv', delimiter=',')
data.drop(columns=data.columns[0], inplace=True)
data.head()

,Id,running_time,completed_time,route_distance_km,delta_time,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,...,finish_cluster_20,finish_cluster_21,finish_cluster_22,finish_cluster_23,finish_cluster_24,finish_cluster_25,finish_cluster_26,finish_cluster_27,finish_cluster_28,finish_cluster_29
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,0,1,0,0,0,...,0.007509,0.002844,0.017119,0.009537,0.014824,0.222392,0.049963,0.183087,0.005667,0.012826
1,-7575630690398473489,2022-01-24 06:53:53,2022-01-24 07:06:26,3.526,753.0,0,0,1,0,0,...,0.004616,0.003340,0.011195,0.006675,0.006387,0.023805,0.012733,0.041042,0.004014,0.024060
2,-6264582368520213833,2022-01-24 10:00:59,2022-01-24 10:15:58,5.071,899.0,0,0,1,0,0,...,0.002925,0.001824,0.004722,0.003344,0.005096,0.020407,0.010571,0.060691,0.002381,0.005569
3,5964315354301636538,2022-01-24 14:28:05,2022-01-24 14:35:08,2.867,423.0,1,0,0,0,0,...,0.001216,0.001089,0.001521,0.001286,0.001676,0.002713,0.002241,0.003440,0.001077,0.001817
4,1372379574816145639,2022-01-24 11:57:29,2022-01-24 12:06:29,3.751,540.0,0,0,1,0,0,...,0.011364,0.024400,0.027831,0.021510,0.009119,0.012878,0.011230,0.013064,0.013189,0.375613


In [3]:
pred_time = pd.read_csv('pred_time.csv', delimiter=',')
pred_time.head()

,Id,pred_time
0,-4773019581999572651,445.785429
1,-7575630690398473489,339.653051
2,-6264582368520213833,555.164520
3,5964315354301636538,327.574647
4,1372379574816145639,410.407676


In [4]:
data = pd.merge(data, pred_time, on='Id', how='inner')

In [5]:
y = data['delta_time']
data.drop(columns=['Id', 'running_time', 'completed_time', 'delta_time'], inplace=True) #'Id', 'running_time', 'completed_time', 

In [6]:
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
X_train, X_valid, y_train, y_valid = train_test_split(data.values, y.values, test_size=0.15, random_state=42)
# X_train_std = sc.fit_transform(data.values)
# X_valid_std = sc.transform(X_valid)

In [12]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

xgb_pars = {'min_child_weight': 1, 'eta': 0.05, 'colsample_bytree': 0.6, 'max_depth': 6,
            'subsample': 0.9, 'lambda': 1., 'alpha': 0.0, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:tweedie', 'tweedie_variance_power':1.25}

In [13]:
model = xgb.train(xgb_pars, dtrain, 2000, watchlist, early_stopping_rounds=300,
                  maximize=False, verbose_eval=10)

[03:45:45] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:649.14392	valid-rmse:647.25928
[10]	train-rmse:645.55780	valid-rmse:643.67395
[20]	train-rmse:623.10394	valid-rmse:621.24756
[30]	train-rmse:539.61407	valid-rmse:538.40668
[40]	train-rmse:403.22986	valid-rmse:404.48639
[50]	train-rmse:279.24548	valid-rmse:284.88403
[60]	train-rmse:193.03255	valid-rmse:204.71957
[70]	train-rmse:140.66029	valid-rmse:159.19551
[80]	train-rmse:111.70848	valid-rmse:136.11124
[90]	train-rmse:96.76317	valid-rmse:126.14156
[100]	train-rmse:88.87301	valid-rmse:121.60725
[110]	train-rmse:84.66350	valid-rmse:119.52010
[120]	train-rmse:81.52792	valid-rmse:118.48451
[130]	train-rmse:79.27904	valid-rm

In [16]:
xgb_pars = {'min_child_weight': 1, 'eta': 0.0005, 'colsample_bytree': 0.5, 'max_depth': 15,
            'subsample': 0.9, 'lambda': 1., 'alpha': 0.0, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:tweedie', 'tweedie_variance_power':1.25}

model2 = xgb.train(xgb_pars, dtrain, 2000, watchlist, early_stopping_rounds=300,
                  maximize=False, verbose_eval=10, xgb_model=model1)

[03:48:48] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:42.26815	valid-rmse:119.49242
[10]	train-rmse:42.20680	valid-rmse:119.49628
[20]	train-rmse:42.14639	valid-rmse:119.49876
[30]	train-rmse:42.09513	valid-rmse:119.50021
[40]	train-rmse:42.04236	valid-rmse:119.50412
[50]	train-rmse:41.98319	valid-rmse:119.50631
[60]	train-rmse:41.93630	valid-rmse:119.50816
[70]	train-rmse:41.87535	valid-rmse:119.51120
[80]	train-rmse:41.83372	valid-rmse:119.51382
[90]	train-rmse:41.76908	valid-rmse:119.51533
[100]	train-rmse:41.71843	valid-rmse:119.51879
[110]	train-rmse:41.66351	valid-rmse:119.52375
[120]	train-rmse:41.60283	valid-rmse:119.52499
[130]	train-rmse:41.53006	valid-rmse:119.52